In [1]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle
import tensorflow

c:\python39\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\python39\lib\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
c:\python39\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout , Activation, Flatten , Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import SGD
import random

In [3]:
words=[]
labels = []
docs = []
ignore_list = ['?', '!']

In [46]:
dataset = open('intents.json').read()
intents = json.loads(dataset)

In [47]:
for intent in intents['intents']:
    for pattern in intent['patterns']:

        #tokenize each word
        word_token = nltk.word_tokenize(pattern)
        words.extend(word_token)
        #add documents in the corpus
        docs.append((word_token, intent['tag']))

        # add to our labels list
        if intent['tag'] not in labels:
            labels.append(intent['tag'])

In [48]:
# lemmatize each word, and sort words by removing duplicates:
words = [lemmatizer.lemmatize(word.lower()) for word in words if word not in ignore_list]
words = sorted(list(set(words)))

In [49]:
# sort labels:
labels = sorted(list(set(labels)))

In [51]:
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(labels,open('labels.pkl','wb'))

In [52]:
# creating our training data:
training_data = []
# creating an empty array for our output (with size same as length of labels):
output = [0]*len(labels)

for doc in docs:
    bag_of_words = []
    pattern_words = doc[0]
    #lemmatize pattern words:
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    
    for w in words:
        if w in pattern_words:
            bag_of_words.append(1)
        else:
            bag_of_words.append(0)
            
    output_row = list(output)
    output_row[labels.index(doc[1])] = 1
    
    training_data.append([bag_of_words,output_row])

In [53]:
# convert training_data to numpy array and shuffle the data:
random.shuffle(training_data)
training_data = np.array(training_data)

<ipython-input-53-d2a7c1a73fe0>:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training_data = np.array(training_data)


In [54]:
# Now we have to create training and testing list:
x_train = list(training_data[:,0])
y_train = list(training_data[:,1])

In [55]:
# Creating Model:

model = Sequential()
model.add(Dense(128, input_shape=(len(x_train[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(y_train[0]), activation='softmax'))

In [56]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_9 (Dense)              (None, 128)               11392     
_________________________________________________________________
dropout_6 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 64)                8256      
_________________________________________________________________
dropout_7 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_11 (Dense)             (None, 9)                 585       
Total params: 20,233
Trainable params: 20,233
Non-trainable params: 0
_________________________________________________________________


In [57]:
sgd_optimizer = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd_optimizer, metrics=['accuracy'])

In [58]:
# fit the model 
history = model.fit(np.array(x_train), np.array(y_train), epochs=200, batch_size=5, verbose=1)

Epoch 1/200
29/29 [==============================] - 1s 7ms/step - loss: 2.1108 - accuracy: 0.2057
Epoch 2/200
29/29 [==============================] - 0s 4ms/step - loss: 1.7313 - accuracy: 0.4113
Epoch 3/200
29/29 [==============================] - 0s 3ms/step - loss: 1.4428 - accuracy: 0.6028
Epoch 4/200
29/29 [==============================] - 0s 2ms/step - loss: 1.0849 - accuracy: 0.7163
Epoch 5/200
29/29 [==============================] - 0s 2ms/step - loss: 0.8034 - accuracy: 0.7730
Epoch 6/200
29/29 [==============================] - 0s 2ms/step - loss: 0.6749 - accuracy: 0.8511
Epoch 7/200
29/29 [==============================] - 0s 2ms/step - loss: 0.4892 - accuracy: 0.8936
Epoch 8/200
29/29 [==============================] - 0s 2ms/step - loss: 0.4247 - accuracy: 0.8936
Epoch 9/200
29/29 [==============================] - 0s 1ms/step - loss: 0.7368 - accuracy: 0.8227
Epoch 10/200
29/29 [==============================] - 0s 1ms/step - loss: 0.4443 - accuracy: 0.8936
Epoch 11/

29/29 [==============================] - 0s 1ms/step - loss: 0.0168 - accuracy: 0.9858
Epoch 164/200
29/29 [==============================] - 0s 1ms/step - loss: 0.0060 - accuracy: 1.0000
Epoch 165/200
29/29 [==============================] - 0s 1ms/step - loss: 0.0026 - accuracy: 1.0000
Epoch 166/200
29/29 [==============================] - 0s 1ms/step - loss: 0.0048 - accuracy: 1.0000
Epoch 167/200
29/29 [==============================] - 0s 1ms/step - loss: 0.0016 - accuracy: 1.0000
Epoch 168/200
29/29 [==============================] - 0s 2ms/step - loss: 0.0159 - accuracy: 0.9929
Epoch 169/200
29/29 [==============================] - 0s 2ms/step - loss: 0.0034 - accuracy: 1.0000
Epoch 170/200
29/29 [==============================] - 0s 2ms/step - loss: 0.0073 - accuracy: 1.0000
Epoch 171/200
29/29 [==============================] - 0s 1ms/step - loss: 0.0044 - accuracy: 1.0000
Epoch 172/200
29/29 [==============================] - 0s 1ms/step - loss: 0.0384 - accuracy: 0.9929
Epoc

In [59]:
model.save('chatbot_Application_model.h5', history)